# Import libraries

In [1]:
import torch
import os

from sympy.matrices.expressions.tests.test_slice import test_entry
from sympy.utilities.tests.test_lambdify import test_integral
from torch.utils.data import DataLoader
from segmentation_models_pytorch.utils.metrics import IoU
from tqdm import tqdm

from src.utils import save_to_hdf5, load_hdf5, load_model, multiclass_iou
from src.datasets import ExcavatorDataset
from src.config import IMAGE_SIZE, TRANSFORMER, DEVICE, ROOT
from models.Segmentation import DeepLabV3Model, DeepLabV3PlusModel, PyramidAttentionNetworkModel, UNetModel

/home/ais/.virtualenvs/similarity_metrics_of_images/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Root

In [2]:
root = ROOT
batch_size = 1

# Initialize models

**Note**: UNEt performs quite badly (only achieves `val IoU` of 0.78).

In [3]:
# DeepLabV3
dlv3 =DeepLabV3Model(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3_HybridFocalDiceLoss.pt')

# DeepLabV3Plus
dlv3p = DeepLabV3PlusModel(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3Plus_HybridFocalDiceLoss.pt')

# UNet
unet = UNetModel(model_path=f'{ROOT}/models/torch_model_files/UNet_HybridFocalDiceLoss.pt')

# Pyramid Attention Network
pan = PyramidAttentionNetworkModel(model_path=f'{ROOT}/models/torch_model_files/PyramidAttentionNetwork_HybridFocalDiceLoss.pt')

# Pass a model to its corresponding ioU file
iou_paths = {
    dlv3: {
        'train': f'{ROOT}/res/model_performance/train_iou_dlv3.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_dlv3.h5'
    },
    dlv3p: {
        'train': f'{ROOT}/res/model_performance/train_iou_dlv3p.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_dlv3p.h5'
    },
    unet: {
        'train': f'{ROOT}/res/model_performance/train_iou_unet.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_unet.h5'
    },
    pan: {
        'train': f'{ROOT}/res/model_performance/train_iou_pan.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_pan.h5'
    }
}

# Confidence paths
confidence_paths = {
    dlv3: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3_no_bg_val.h5'
        }
    },
    dlv3p: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3p_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3p_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3p_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3p_no_bg_val.h5'
        }
    },
    unet: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/unet_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/unet_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/unet_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/unet_no_bg_val.h5'
        }
    },
    pan: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/pan_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/pan_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/pan_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/pan_no_bg_val.h5'
        }
    }
}

2024-12-15 08:16:24,864 - DeepLabV3 - INFO - Device used for model: cuda
2024-12-15 08:16:25,178 - DeepLabV3Plus - INFO - Device used for model: cuda
2024-12-15 08:16:25,515 - UNet - INFO - Device used for model: cuda
2024-12-15 08:16:25,833 - PyramidAttentionNetwork - INFO - Device used for model: cuda


# Load dataset

In [4]:
train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', transform=TRANSFORMER)
print("Number of training samples:", num_train_imgs:=len(train_dataset))
test_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test', transform=TRANSFORMER)
print("Number of test samples:", num_test_imgs:=len(test_dataset))

Number of training samples: 1782
Number of test samples: 187


/home/ais/Bachelorarbeit/similarity_metrics_of_images/src/datasets.py:308: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(value / 255.0, dtype=torch.float32)


# Compute and save predicted masks

In [9]:
train_iou_dlv3 = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_iou_dlv3p = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_iou_unet = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_iou_pan = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_paths = []

test_iou_dlv3 = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_iou_dlv3p = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_iou_unet = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_iou_pan = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_paths = []

In [10]:

# Compute predicted masks for training set
for i, (imgs, masks, paths) in tqdm(enumerate(train_dataset)):
    imgs = imgs.to(DEVICE).unsqueeze(0)
    masks = masks.to(DEVICE)
    output_dlv3 = dlv3(imgs).squeeze(0)
    output_dlv3p = dlv3p(imgs).squeeze(0)
    output_unet = unet(imgs).squeeze(0)
    output_pan = pan(imgs).squeeze(0)
    train_iou_dlv3[i] = multiclass_iou(output_dlv3, masks)
    train_iou_dlv3p[i] = multiclass_iou(output_dlv3p, masks)
    train_iou_unet[i] = multiclass_iou(output_unet, masks)
    train_iou_pan[i] = multiclass_iou(output_pan, masks)
    train_paths.append(paths)

train_paths = [path.replace('|', '/') for path in train_paths]
save_to_hdf5(f'{root}/res/model_performance/train_iou_dlv3.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_dlv3)})
save_to_hdf5(f'{root}/res/model_performance/train_iou_dlv3p.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_dlv3p)})
save_to_hdf5(f'{root}/res/model_performance/train_iou_unet.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_unet)})
save_to_hdf5(f'{root}/res/model_performance/train_iou_pan.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_pan)})

# Compute predicted masks for validation set
for i, (imgs, masks, paths) in tqdm(enumerate(test_dataset)):
    imgs = imgs.to(DEVICE).unsqueeze(0)
    masks = masks.to(DEVICE)
    output_dlv3 = dlv3(imgs).squeeze(0)
    output_dlv3p = dlv3p(imgs).squeeze(0)
    output_unet = unet(imgs).squeeze(0)
    output_pan = pan(imgs).squeeze(0)
    test_iou_dlv3[i] = multiclass_iou(output_dlv3, masks)
    test_iou_dlv3p[i] = multiclass_iou(output_dlv3p, masks)
    test_iou_unet[i] = multiclass_iou(output_unet, masks)
    test_iou_pan[i] = multiclass_iou(output_pan, masks)
    test_paths.append(paths)

test_paths = [path.replace('|', '/') for path in test_paths]
save_to_hdf5(f'{root}/res/model_performance/val_iou_dlv3.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_dlv3)})
save_to_hdf5(f'{root}/res/model_performance/val_iou_dlv3p.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_dlv3p)})
save_to_hdf5(f'{root}/res/model_performance/val_iou_unet.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_unet)})
save_to_hdf5(f'{root}/res/model_performance/val_iou_pan.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_pan)})


1782it [00:53, 33.43it/s]
187it [00:05, 34.98it/s]


# Compute and save Prediction Confidence for all models

In [6]:
def compute_and_save_confidence_vectors(model, train_dataset, val_dataset, iou_paths):
    """
    Compute and save prediction probability vectors for train and val sets.
    """
    model_name = model.model.__class__.__name__
    out_paths = iou_paths[model]  # {'train': ..., 'val': ...}

    # Create directories if they don't exist
    os.makedirs(os.path.dirname(out_paths['with_background']['train']), exist_ok=True)
    os.makedirs(os.path.dirname(out_paths['with_background']['val']), exist_ok=True)
    os.makedirs(os.path.dirname(out_paths['no_background']['train']), exist_ok=True)
    os.makedirs(os.path.dirname(out_paths['no_background']['val']), exist_ok=True)

    # We'll save dictionaries first, then write to HDF5
    train_results_with_bg = {}
    train_results_no_bg = {}
    val_results_with_bg = {}
    val_results_no_bg = {}

    # Process Train Set
    for img, mask, path in tqdm(train_dataset, desc=f"Computing confidence vectors (Train) for {model_name}"):
        # Get probabilities with background
        conf_with_bg, _ = model.predict_single_image(img, mask,                                          return_raw_prob_vector=True,                                             ignore_background=False)
        # Get probabilities without background
        conf_no_bg, _ = model.predict_single_image(img, mask,
                                                   return_raw_prob_vector=True,
                                                   ignore_background=True)
        train_results_with_bg[os.path.basename(path)] = conf_with_bg.cpu().numpy()
        train_results_no_bg[os.path.basename(path)] = conf_no_bg.cpu().numpy()

    # Process Val Set
    for img, mask, path in tqdm(val_dataset, desc=f"Computing confidence vectors (Val) for {model_name}"):
        # Get probabilities with background
        conf_with_bg, _ = model.predict_single_image(img, mask,
                                                     return_raw_prob_vector=True,
                                                     ignore_background=False)
        # Get probabilities without background
        conf_no_bg, _ = model.predict_single_image(img, mask,
                                                   return_raw_prob_vector=True,
                                                   ignore_background=True)
        val_results_with_bg[os.path.basename(path)] = conf_with_bg.cpu().numpy()
        val_results_no_bg[os.path.basename(path)] = conf_no_bg.cpu().numpy()

    # Save results
    save_to_hdf5(out_paths['with_background']['train'], train_results_with_bg)
    save_to_hdf5(out_paths['with_background']['val'], val_results_with_bg)
    save_to_hdf5(out_paths['no_background']['train'], train_results_no_bg)
    save_to_hdf5(out_paths['no_background']['val'], val_results_no_bg)

# Run for all models
compute_and_save_confidence_vectors(dlv3, train_dataset, test_dataset, confidence_paths)
compute_and_save_confidence_vectors(dlv3p, train_dataset, test_dataset, confidence_paths)
compute_and_save_confidence_vectors(unet, train_dataset, test_dataset, confidence_paths)
compute_and_save_confidence_vectors(pan, train_dataset, test_dataset, confidence_paths)

Computing confidence vectors (Train) for DeepLabV3:   0%|          | 0/1782 [00:00<?, ?it/s]/home/ais/.virtualenvs/similarity_metrics_of_images/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
Computing confidence vectors (Train) for DeepLabV3: 100%|██████████| 1782/1782 [01:24<00:00, 21.12it/s]
Computing confidence vectors (Val) for DeepLabV3: 100%|██████████| 187/187 [00:08<00:00, 21.11it/s]
Computing confidence vectors (Train) for DeepLabV3Plus: 100%|██████████| 1782/1782 [01:21<00:00, 21.94it/s]
Computing confidence vectors (Val) for DeepLabV3Plus: 100%|██████████| 187/187 [00:07<00:00, 24.37it/s]
Computing confidence vectors (Val) for PAN: 100%|██████████| 187/187 [00:08<00:00, 21.01it/s]


## Check that data is loaded correctly

Shape of train_iou: 1782
Shape of val_iou: 187
